


# Superhero and Supervillain Name Generator

---

[Superhero Names Dataset](https://github.com/am1tyadav/superhero)

In [ ]:
!git clone https://github.com/am1tyadav/superhero

Cloning into 'superhero'...
remote: Enumerating objects: 8, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 8 (delta 0), reused 4 (delta 0), pack-reused 0
Unpacking objects: 100% (8/8), done.


In [ ]:
with open('superhero/superheroes.txt', 'r') as f:
  data = f.read()

data[:100]


'jumpa\t\ndoctor fate\t\nstarlight\t\nisildur\t\nlasher\t\nvarvara\t\nthe target\t\naxel\t\nbattra\t\nchangeling\t\npyrrh'

In [1]:
import tensorflow as tf

In [ ]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(
    filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~',
    split='\n',
)

In [ ]:
tokenizer.fit_on_texts(data)

In [ ]:
char_to_index = tokenizer.word_index
index_to_char = dict((v, k) for k, v in char_to_index.items())

print(index_to_char)

{1: '\t', 2: 'a', 3: 'e', 4: 'r', 5: 'o', 6: 'n', 7: 'i', 8: ' ', 9: 't', 10: 's', 11: 'l', 12: 'm', 13: 'h', 14: 'd', 15: 'c', 16: 'u', 17: 'g', 18: 'k', 19: 'b', 20: 'p', 21: 'y', 22: 'w', 23: 'f', 24: 'v', 25: 'j', 26: 'z', 27: 'x', 28: 'q'}




Converting between names and sequences

In [ ]:
names = data.splitlines()
names[:10]

['jumpa\t',
 'doctor fate\t',
 'starlight\t',
 'isildur\t',
 'lasher\t',
 'varvara\t',
 'the target\t',
 'axel\t',
 'battra\t',
 'changeling\t']

In [ ]:
tokenizer.texts_to_sequences(names[0])

[[25], [16], [12], [20], [2], [1]]

In [ ]:
def name_to_seq(name):
  return [tokenizer.texts_to_sequences(c)[0][0] for c in name]

In [ ]:
name_to_seq(names[0])

[25, 16, 12, 20, 2, 1]

In [ ]:
def seq_to_name(seq):
  return ''.join([index_to_char[i] for i in seq if i != 0])

In [ ]:
seq_to_name(name_to_seq(names[0]))

'jumpa\t'



1. Creating sequences
2. Padding all sequences

In [ ]:
sequences = []
for name in names:
  seq = name_to_seq(name)
  if len(seq) >= 2:
    sequences += [seq[:i]for i in range(2, len(seq) + 1)]
    

In [ ]:
sequences[:10]

[[25, 16],
 [25, 16, 12],
 [25, 16, 12, 20],
 [25, 16, 12, 20, 2],
 [25, 16, 12, 20, 2, 1],
 [14, 5],
 [14, 5, 15],
 [14, 5, 15, 9],
 [14, 5, 15, 9, 5],
 [14, 5, 15, 9, 5, 4]]

In [ ]:
max_len = max([len(x) for x in sequences])
print(max_len)

33


In [ ]:
padded_sequences = tf.keras.preprocessing.sequence.pad_sequences(
    sequences, padding='pre',
    maxlen= max_len
)

print(padded_sequences[0])

[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0 25 16]


In [ ]:
padded_sequences.shape

(88279, 33)



Creating training and validation sets

In [ ]:
x, y = padded_sequences[:, :-1], padded_sequences[:, -1]
print(x.shape, y.shape)

(88279, 32) (88279,)


In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y)

print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(66209, 32) (66209,)
(22070, 32) (22070,)


In [ ]:
num_chars = len(char_to_index.keys()) + 1
print(num_chars)

29


Creating the Model

In [ ]:
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, MaxPool1D, LSTM
from tensorflow.keras.layers import Bidirectional, Dense

In [ ]:


model = Sequential([
                    Embedding(num_chars, 8, input_length = max_len-1),
                    Conv1D(64, 5, strides=1, activation='tanh', padding='causal'),
                    MaxPool1D(2),
                    LSTM(32),
                    Dense(num_chars, activation='softmax')
])

model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 32, 8)             232       
                                                                 
 conv1d_4 (Conv1D)           (None, 32, 64)            2624      
                                                                 
 max_pooling1d_4 (MaxPooling  (None, 16, 64)           0         
 1D)                                                             
                                                                 
 lstm_4 (LSTM)               (None, 32)                12416     
                                                                 
 dense_2 (Dense)             (None, 29)                957       
                                                                 
Total params: 16,229
Trainable params: 16,229
Non-trainable params: 0
__________________________________________________

Training the Model

In [ ]:

h = model.fit(
    x_train, y_train,
    validation_data = (x_test, y_test),
    epochs=50, verbose=2,
    callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=3)
    ])


Epoch 1/50
2070/2070 - 32s - loss: 2.7590 - accuracy: 0.1846 - val_loss: 2.5869 - val_accuracy: 0.2203 - 32s/epoch - 15ms/step
Epoch 2/50
2070/2070 - 17s - loss: 2.5489 - accuracy: 0.2313 - val_loss: 2.5092 - val_accuracy: 0.2408 - 17s/epoch - 8ms/step
Epoch 3/50
2070/2070 - 17s - loss: 2.4852 - accuracy: 0.2492 - val_loss: 2.4598 - val_accuracy: 0.2522 - 17s/epoch - 8ms/step
Epoch 4/50
2070/2070 - 17s - loss: 2.4402 - accuracy: 0.2624 - val_loss: 2.4216 - val_accuracy: 0.2665 - 17s/epoch - 8ms/step
Epoch 5/50
2070/2070 - 17s - loss: 2.4028 - accuracy: 0.2719 - val_loss: 2.3933 - val_accuracy: 0.2737 - 17s/epoch - 8ms/step
Epoch 6/50
2070/2070 - 17s - loss: 2.3709 - accuracy: 0.2814 - val_loss: 2.3673 - val_accuracy: 0.2791 - 17s/epoch - 8ms/step
Epoch 7/50
2070/2070 - 17s - loss: 2.3436 - accuracy: 0.2896 - val_loss: 2.3420 - val_accuracy: 0.2864 - 17s/epoch - 8ms/step
Epoch 8/50
2070/2070 - 17s - loss: 2.3188 - accuracy: 0.2973 - val_loss: 2.3306 - val_accuracy: 0.2897 - 17s/epoch - 

Generate Names!

In [ ]:
from math import trunc
def generate_names(seed):
  for i in range(0, 40):
    seq = name_to_seq(seed)
    padded = tf.keras.preprocessing.sequence.pad_sequences([seq], padding='pre', maxlen=max_len-1, truncating='pre'
    )

    pred = model.predict(padded)[0]
    pred_char = index_to_char[tf.argmax(pred).numpy()]
    seed += pred_char
 
    if pred_char == '\t':
      break
  print(seed)

In [ ]:

generate_names('f')

frack man	
